# Calculate Image Histogram

In this example, we will compute the local and global image statistics

## Setup NVFLARE

follow the [Getting_Started](https://nvflare.readthedocs.io/en/main/getting_started.html) to setup virtual environment and install NVFLARE

You can also follow this [Getting Started](../../../../getting_started.ipynb) Notebook to setup. 

Assume you have already setup the venv, lets first install required packages.
First find out which directory we are 


In [ ]:
 !pwd

## Install requirements

In [ ]:
!pip install -r image_stats/requirements.txt

## Download data

As an example, we use the dataset from the ["COVID-19 Radiography Database"](https://www.kaggle.com/tawsifurrahman/covid19-radiography-database).
it contains png image files in four different classes: `COVID`, `Lung_Opacity`, `Normal`, and `Viral Pneumonia`.
First, download and extract to `/tmp/nvflare/image_stats/data/.`.


In [ ]:
ls -l /tmp/nvflare/image_stats/data/.


## Prepare data

Next, create the data lists simulating different clients with varying amounts and types of images. 
The downloaded archive contains subfolders for four different classes: `COVID`, `Lung_Opacity`, `Normal`, and `Viral Pneumonia`.
Here we assume each class of image corresponds to a different sites.




In [ ]:
! image_stats/prepare_data.sh


## Run job in FL Simulator

With FL simulator, we can just run the example with CLI command 



In [ ]:
! nvflare simulator image_stats/jobs/image_stats -w /tmp/nvflare/image_stats -n 4 -t 4



The results are stored in workspace "/tmp/nvflare/image_stats"

In [ ]:
! ls -al /tmp/nvflare/image_stats/simulate_job/statistics/image_statistics.json

## Visualization
We can visualize the results easly via the visualizaiton notebook. Before we do that, we need to copy the data to the notebook directory 


In [ ]:
! cp /tmp/nvflare/image_stats/simulate_job/statistics/image_statistics.json image_stats/demo/.

now we can visualize via the [visualization notebook](image_stats/demo/visualization.ipynb)

We are not quite done yet. What if you prefer to use python API instead CLI to run jobs. Lets do that in this section

## Run Job using Simulator API
This should be the same as running in command CLI via nvflare simulator

In [ ]:
from nvflare.private.fed.app.simulator.simulator_runner import SimulatorRunner
runner = SimulatorRunner(job_folder="image_stats/jobs/image_stats", workspace="/tmp/nvflare/image_stats", n_clients = 4, threads=4)
runner.run()

## Run Job using FLARE API

So far, we having using Simulator to simulate the federated job run. With [FLARE API](../../../tutorial/flare_api.ipynb) , you can directly interact with NVFLARE system in production or POC mode
In such cases, we have to first setup and deploy the federated system. Since we are running on local machine, we will minic this deploy via Proof of Conccept mode.
Please refer this section to see how to [setup POC mode](../../../tutorial/setup_poc.ipynb)
You double check if the flare is running with the following command from **terminal**


In [ ]:
! ps -eaf | grep nvflare

###
If you determine that the flare poc system is not running, you can open a terminal to start the FLARE system in POC mode. 
You need to read more about POC, check [setup POC mode](../../../tutorial/setup_poc.ipynb)

At this point, assume you have already setup the poc and started the NVFLARE. And we are going to use the default **workspace=/tmp/nvflare/poc**. We will first check the system status

In [ ]:
import os
from nvflare.fuel.flare_api.flare_api import new_insecure_session

workspace = "/tmp/nvflare/poc"
admin_dir = os.path.join(workspace, "admin")
sess = new_insecure_session(admin_dir)
print(sess.get_system_info())


###
**submit Job**

In [ ]:
examples_dir = os.path.join(admin_dir, "transfer")
job_folder = os.path.join(examples_dir, "advanced/federated-statistics/image_stats/jobs/image_stats")
job_id = sess.submit_job(job_folder)
print(job_id + " was submitted")

###

**Monitoring Job**

You can choose your monitoring output, here is one function to display the job information 

In [ ]:
from nvflare.fuel.flare_api.flare_api import Session
def status_monitor_cb(
        session: Session, job_id: str, job_meta, *cb_args, **cb_kwargs
    ) -> bool:
    if job_meta["status"] == "RUNNING":
        if cb_kwargs["cb_run_counter"]["count"] < 3 or cb_kwargs["cb_run_counter"]["count"]%20 == 0:
            print(job_meta)            
        else: 
            print(".", end="")
    else:
        print("\n" + str(job_meta))
    
    cb_kwargs["cb_run_counter"]["count"] += 1
    return True


In [ ]:
sess.get_job_meta(job_id)

In [ ]:
sess.monitor_job(job_id, cb=status_monitor_cb, cb_run_counter={"count":0})

In [ ]:
import json

def format_json( data: dict): 
    print(json.dumps(data, sort_keys=True, indent=4,separators=(',', ': ')))


list_jobs_output = sess.list_jobs(limit = 2)
print( format_json(list_jobs_output))

In [ ]:
list_jobs_output_detailed = sess.list_jobs(detailed=True)
print(format_json(list_jobs_output_detailed))

###

**Download the result from FL Server**

In [ ]:
result_dir = sess.download_job_result(job_id)
print(result_dir)

In [ ]:
! tree /tmp/nvflare/poc/admin/transfer/953e0a62-c00a-4a69-bd81-aa00309373f2

## logs

In [ ]:
!head -n 100 /tmp/nvflare/poc/admin/transfer/14a836f6-ae77-4971-91c0-d14f837df5fb/workspace/log.txt

## Visualization

Now we can copy the adults_stats.json to the demo folder for visualization

In [ ]:
import shutil
import os
src= os.path.join(result_dir, workspace/statistics

# 2nd option
shutil.copy(src, dst)


! cp  /tmp/nvflare/poc/admin/transfer/093d5524-761f-433d-8823-9977dbdd0f90/workspace/statistics/adults_stats.json df_stats/demo/.

Now we can visualize via the [visualization notebook](df_stats/demo/visualization.ipynb) as before


## Cleanup
If you like to clean up the temp folders and POC, we need some clean up
* remove downloaded result folder 
* shutdown POC 
* clean up POC workspace

In [ ]:
! rm -r /tmp/nvflare/poc/admin/transfer/093d5524-761f-433d-8823-9977dbdd0f90

In [ ]:
! nvflare poc --stop

In [ ]:
!nvflare poc --clean

## We are done !
Congratulations, you just completed the federated stats image histogram calulation
